In [1]:
import numpy as np
from scipy.integrate import solve_ivp

In [2]:
def smoothabs(x, xi=10000):
    """Smooth absolute value function"""
    return x * np.tanh(x * xi)


In [3]:
 def gottwald_noice(t, u, p, stochsys=True):
    """Gottwald model functions without sea ice"""
    x, y, z, T, S = u
    if stochsys:
        pvec = p[0] if isinstance(p, list) else p
    else:
        pvec = p
    
    (a, b, mu, epsilon_a, epsilon_f, F0, F1, G0, G1, 
     theta_0, theta_1, sigma_0, sigma_1, x_mean, Delta_mean) = pvec
    
    Delta = y**2 + z**2
    T_surf = theta_0 + theta_1 * (x - x_mean)/(np.sqrt(epsilon_f))
    S_surf = sigma_0 + sigma_1 * (Delta - Delta_mean)/(np.sqrt(epsilon_f))
    
    dx = 1/epsilon_f*(-Delta - a*(x - F0 - F1*T))
    dy = 1/epsilon_f*(x*y - b*x*z - (y - G0 + G1*T))
    dz = 1/epsilon_f*(b*x*y + x*z - z)
    dT = -1/epsilon_a*(T - T_surf) - T - mu*smoothabs(S-T)*T
    dS = S_surf - S - mu*smoothabs(S-T)*S
    
    return np.array([dx, dy, dz, dT, dS])

In [4]:
def gottwald_noice_scaled(t, u, p, stochsys=True):
    """Rescaled (x/y/z) for edge tracking"""
    x, y, z, T, S = u
    if stochsys:
        pvec = p[0] if isinstance(p, list) else p
    else:
        pvec = p
    
    (a, b, mu, epsilon_a, epsilon_f, F0, F1, G0, G1, 
     theta_0, theta_1, sigma_0, sigma_1, x_mean, Delta_mean, s) = pvec
    
    Delta = y**2 + z**2
    T_surf = theta_0 + theta_1 * (s*x - x_mean)/(np.sqrt(epsilon_f))
    S_surf = sigma_0 + sigma_1 * (s**2*Delta - Delta_mean)/(np.sqrt(epsilon_f))
    
    dx = 1/epsilon_f*(-s*Delta - a/s*(s*x - F0 - F1*T))
    dy = 1/epsilon_f*(s*x*y - s*b*x*z - (s*y - G0 + G1*T)/s)
    dz = 1/epsilon_f*(s*b*x*y + s*x*z - z)
    dT = -1/epsilon_a*(T - T_surf) - T - mu*smoothabs(S-T)*T
    dS = S_surf - S - mu*smoothabs(S-T)*S
    
    return np.array([dx, dy, dz, dT, dS])

In [5]:
def gottwald_noice_seasons(t, u, p):
    """Gottwald model functions without sea ice, but with seasonal cycle"""
    x, y, z, T, S = u
    (a, b, mu, epsilon_a, epsilon_f, F0, F1, G0, G1, 
     theta_0, theta_1, sigma_0, sigma_1, x_mean, Delta_mean, td, F2, G2) = p[0]
    
    omega = 2.0 * np.pi * td  # annual frequency
    Delta = y**2 + z**2
    T_surf = theta_0 + theta_1 * (x - x_mean)/(np.sqrt(epsilon_f))
    S_surf = sigma_0 + sigma_1 * (Delta - Delta_mean)/(np.sqrt(epsilon_f))
    
    dx = 1/epsilon_f*(-Delta - a*(x - F0 - F1*T - F2*np.cos(omega * t)))
    dy = 1/epsilon_f*(x*y - b*x*z - (y - G0 + G1*T - G2*np.cos(omega * t)))
    dz = 1/epsilon_f*(b*x*y + x*z - z)
    dT = -1/epsilon_a*(T - T_surf) - T - mu*smoothabs(S-T)*T
    dS = S_surf - S - mu*smoothabs(S-T)*S
    
    return np.array([dx, dy, dz, dT, dS])

In [6]:
def param_gwn_default(param_l84=None, sigma_0=0.926): #**kwargs):
    """return parameter values"""
    if param_l84 is not None:
        a, b, F0, G0 = param_l84
    else:
        param_l84 = [0.25, 4., 8., 1.]  # default L84 params [a, b, F0, G0]
        
    x_std = 0.513
    Delta_std = 1.071
    x_mean = 1.0147
    Delta_mean = 1.7463  # long-run results from default L84
    
    # Unpack L84 params
    a, b, F0, G0 = param_l84
    
    # Other default model params
    mu = 7.5
    theta_0 = 1.
    # sigma_0 = 0.9  # Stommel params (is varied) 0.926, 0.932
    epsilon_a = 0.34
    epsilon_f = 0.0003  # 0.0083 # timescales
    F1 = 0.1
    G1 = 0.  # coupling params
    perturb_scaling = 0.01  # coupling strength of L84 to Stommel
    
    # Coupling params from L84 run/default
    theta_1 = min(theta_0, perturb_scaling/x_std)
    sigma_1 = min(sigma_0, perturb_scaling/Delta_std)
   
    
    return [a, b, mu, epsilon_a, epsilon_f, F0, F1, G0, G1, 
            theta_0, theta_1, sigma_0, sigma_1, x_mean, Delta_mean]

In [7]:
def simulate_gottwald_noice(initial_conditions, params, t_span, t_eval=None, stochsys=True):
    """Simulate the Gottwald model without sea ice"""
    sol = solve_ivp(
        lambda t, y: gottwald_noice(t, y, params, stochsys),
        t_span,
        initial_conditions,
        t_eval=t_eval,
        method='RK45',
        rtol=1e-10,
        atol=1e-10
    )
    return sol

In [8]:
# Example usage
if __name__ == "__main__":
    import matplotlib.pyplot as plt
    import matplotlib
    matplotlib.use('Agg')
    
    # Get default Gottwald parameters
    params_gwn = param_gwn_default()
    print(f"Default Gottwald parameters:")
    print(params_gwn)
    
    # Simulate Gottwald model
    initial_gwn = [1.0, 0.5, 0.5, 1.0, 0.9]  # [x, y, z, T, S]
    # params_gwn_wrapped = [params_gwn]  # Wrap in list for stochsys format
    tmax = 50 # 00
    t_eval = np.linspace(0, tmax, tmax* 10000)

    sol_gwn = simulate_gottwald_noice(initial_gwn, params_gwn, (0, tmax), 
                                       t_eval=t_eval, stochsys=False)



    fig, ax = plt.subplots(1, 3, figsize=(4 * 2.5, 3))
    ax[0].plot(sol_gwn.t, sol_gwn.y[3], label="T")
    ax[0].plot(sol_gwn.t, sol_gwn.y[4], label="S")
    ax[0].legend()
    ax[0].set_xlabel("t")
    ax[1].plot(sol_gwn.y[0][:1000], sol_gwn.y[1][:1000])
    ax[1].plot(sol_gwn.y[0][-1000:], sol_gwn.y[1][-1000:])
    ax[1].set_xlabel("x")
    ax[1].set_ylabel("y")
    ax[2].plot(sol_gwn.y[1][:1000], sol_gwn.y[2][:1000])
    ax[2].plot(sol_gwn.y[1][-1000:], sol_gwn.y[2][-1000:])    
    ax[2].set_xlabel("y")
    ax[2].set_ylabel("z")
    fig.tight_layout()   

    fig.savefig('gottwald_noice.png')

Default Gottwald parameters:
[0.25, 4.0, 7.5, 0.34, 0.0003, 8.0, 0.1, 1.0, 0.0, 1.0, 0.01949317738791423, 0.926, 0.009337068160597572, 1.0147, 1.7463]
